## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report,confusion_matrix, plot_confusion_matrix
from statsmodels.stats.proportion import proportions_ztest,confint_proportions_2indep

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import scipy
import statsmodels

from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import gower

from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.preprocessing import normalize, StandardScaler

from matplotlib import gridspec
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

In [2]:
'''
defining function to bucket statue to crime type, segmented by county and crime type
'''

def sp_county_crime_type(county, crime_type, df):
    match_ind = ''
    if crime_type == 'drug':     ### Expendable
        match_ind = '893.*'
    elif crime_type =='robbery':
        match_ind = '812.*'
    elif crime_type == 'burglary':
        match_ind =='810.*'
    elif crime_type == 'driving':
        match_ind == '322.*'
    
    df = df[df['statut'].str.match(match_ind)== True]
    
    df =df[df['county']== county]
    
    return df

In [3]:
'''
defining function to bucket statue to crime type, segmented by circuit and crime type
'''

def sp_circuit_crime_type(circuit, crime_type, df):
    match_ind = ''
    if crime_type == 'drug':     ### Expendable
        match_ind = '893.*'
    elif crime_type =='robbery':
        match_ind = '812.*'
    elif crime_type == 'burglary':
        match_ind =='810.*'
    elif crime_type == 'driving':
        match_ind == '322.*'
    df = df[df['statut'].str.match(match_ind)== True]
    
    df =df[df['circuit']== circuit]
    
    return df

In [4]:
'''
Preparing clustering by: 
    # generate gender indicator
    # standardize and normalize total points and age feature
'''

def cluster_prep(df):
    df_temp = df.copy()
    df_temp = df_temp[(df_temp['race']=='WHITE')|(df_temp['race']=='BLACK')]
    df_temp['gender_ind']=[1 if x=='MALE' else 0 for x in df_temp['gender']]
    #df_temp = df_temp.loc[:,['gender_ind','gender','age','totpts','race','sp_total_days']] 
    
    X_continous = df_temp.loc[:,['totpts','age']]
    X_cont_stand =StandardScaler().fit_transform(X_continous)
    X_normalized = normalize(X_cont_stand)
    
    df_temp['totpts_stand'] = X_normalized[:,0]
    df_temp['age_stand'] = X_normalized[:,1]
    
    return df_temp

In [5]:
'''
functions on two sample unequal variance t test calculation
Outputing the area and crime type, p-value, confidence interval related to the calculation
'''

import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t
import pandas as pd

def welch_ttest(x1, x2,alternative,area, crime):
    
    n1 = x1.size
    n2 = x2.size
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    
    v1 = np.var(x1, ddof=1)
    v2 = np.var(x2, ddof=1)
    
    pooled_se = np.sqrt(v1 / n1 + v2 / n2)
    delta = m1-m2
    
    tstat = delta /  pooled_se
    df = (v1 / n1 + v2 / n2)**2 / (v1**2 / (n1**2 * (n1 - 1)) + v2**2 / (n2**2 * (n2 - 1)))
    
    # two side t-test
    p = 2 * t.cdf(-abs(tstat), df)
    
    # upper and lower bounds
    lb = delta - t.ppf(0.975,df)*pooled_se 
    ub = delta + t.ppf(0.975,df)*pooled_se
    
    df_result = pd.DataFrame(np.array([tstat,df,p,delta,lb,ub]).reshape(1,-1),
                         columns=['T statistic','df','pvalue 2 sided','Difference in mean','lb','ub'])
    
    
    df_result['area']=area
    df_result['crime'] = crime
    
    return df_result

In [6]:
'''
finding the most fitting number of clusters for K-means clustering
using the "kneed" package to find the optimal k based on inertia
provide flexibility of increasing or decreasing k based on business understanding
'''

def find_k(df, increment=0, decrement=0):
    """Find the optimum k clusters"""
    sse = {}
    
    for k in range(2, 21):
        kmeans = KMeans(n_clusters=k, random_state=1)
        kmeans.fit(df)
        sse[k] = kmeans.inertia_
    
    kn = KneeLocator(x=list(sse.keys()), 
                 y=list(sse.values()), 
                 curve='convex', 
                 direction='decreasing')
    k = kn.knee + increment - decrement
    return k

In [7]:
'''
assign group labels based on k means cluster
cluster is based on standardized totpts and age features
'''

def assign_label(df, k = -1):
    df_temp = df
    X_normalized =df_temp.loc[:,['totpts_stand','age_stand']]
    
    if k == -1:
        k_temp = find_k(X_normalized)
    else:
        k_temp = k
    
    #print("cluster number: ",k_temp)
    kmeanModel = KMeans(n_clusters=k_temp).fit(X_normalized)
    
    df_temp['Y']=kmeanModel.labels_
    
    return df_temp

In [8]:
'''
Sampling with equal weight, the minimum sample size that is usable
Sampling gender group individually then concatenate
'''

def cluster_sampling(df, sample_rate = 0.8):
    
    clusters = dict(df['Y'].value_counts()).keys()
    
    b = []
    w = []
    
    l_sample_size = []
    new_clusters = []
    
    for i in clusters:
        d_temp = dict(df[df['Y']==i]['race'].value_counts())
        sam_size = round(min(d_temp.values())*sample_rate) 
        ### making sure gender wise, the number of samples are the same
        
        if sam_size <30: # check if sample size is smaller than 30, if so, do not sample from this cluster
            continue     # based on observation, this does not happen often
        
        new_clusters.append(i)
        l_sample_size.append(sam_size)
    
    if len(l_sample_size)==0:
        #print('not enough sample to make inference')
        
        b_df = pd.DataFrame()
        w_df = pd.DataFrame()
        return b_df,w_df
    
    min_sam_size = round(min(l_sample_size)) 
    ### ensuring sample extracted are the same from different clusters
    
    for i in new_clusters:
        d_temp = dict(df[df['Y']==i]['race'].value_counts())
        #print(d_temp.items())
        #print('cluster',d_temp)
        
        w_temp = df[(df['Y']==i)&(df['race']=='WHITE')].sample(min_sam_size)
        b_temp = df[(df['Y']==i)&(df['race']=='BLACK')].sample(min_sam_size)
        
        b.append(b_temp)
        w.append(w_temp)
        
    
    b_df = pd.concat(b)
    w_df = pd.concat(w)
    
    return b_df,w_df

In [4]:
''' 
Sampling with equal weight, the minimum sample size that is usable
Sampling gender group individually then concatenate
'''

def cluster_sampling_unequal_strat(df, sample_rate = 0.8):
    
    clusters = dict(df['Y'].value_counts()).keys()
    
    b = []
    w = []
    
    l_sample_size = []
    new_clusters = []
    
    for i in clusters:
        d_temp = dict(df[df['Y']==i]['race'].value_counts())
        sam_size = round(min(d_temp.values())*sample_rate) 
        ### making sure gender wise, the number of samples are the same
        
        if sam_size <30:
            continue
        
        new_clusters.append(i)
        l_sample_size.append(sam_size)
    
    if len(l_sample_size)==0:
        #print('not enough sample to make inference')
        
        b_df = pd.DataFrame()
        w_df = pd.DataFrame()
        return b_df,w_df
    
    #min_sam_size = round(min(l_sample_size)) 
    ### ensuring sample extracted are the same from different clusters
    
    for i in range(len(new_clusters)):
        d_temp = dict(df[df['Y']==new_clusters[i]]['race'].value_counts())
        #print(d_temp.items())
        #print('cluster',d_temp)
        
        w_temp = df[(df['Y']==new_clusters[i])&(df['race']=='WHITE')].sample(round(l_sample_size[i]))
        b_temp = df[(df['Y']==new_clusters[i])&(df['race']=='BLACK')].sample(round(l_sample_size[i]))
        
        b.append(b_temp)
        w.append(w_temp)
        
    
    b_df = pd.concat(b)
    w_df = pd.concat(w)
    
    return b_df,w_df

In [10]:
raw = pd.read_csv("../robling/sentencing_post_eda.csv")

In [11]:
circuit_l =['CIRCUIT 06 - CLEARWATER','CIRCUIT 17 - FT. LAUDERDALE','CIRCUIT 11 - MIAMI']

In [12]:
crime_l = ['drug','robbery','burglary','driving']

In [13]:
result_list = []

for i in crime_l:
    for j in circuit_l:
        print(i,j)
        df_data = sp_circuit_crime_type(circuit=j,crime_type=i,df= raw)
        df_model = cluster_prep(df_data)
        
        gen_male = df_model.loc[df_model['gender_ind']==1,:].copy()
        gen_female = df_model.loc[df_model['gender_ind']==0,:].copy()
        m = assign_label(gen_male)
        f = assign_label(gen_female)
        
        for k in range(1,101): ## instead of running once, run multiple test to provide average test stats
            
            b_m,w_m = cluster_sampling(m)

            ind = 0

            if b_m.shape[0]==0 and w_m.shape[0] == 0:
                #print("not enough male samples")
                ind +=1


            b_f,w_f = cluster_sampling(f)

            if b_f.shape[0]==0 and w_f.shape[0] == 0:
                #print("not enough female samples")
                ind +=1

            if ind == 2:
                #print('not enough samples to make inference')
                continue


            treat = pd.concat([b_m,b_f])
            cont = pd.concat([w_m,w_f])

            temp_result = welch_ttest(treat.sp_total_days, cont.sp_total_days,'unequal',area = j,crime = i)

            #print(temp_result)

            result_list.append(temp_result)

drug CIRCUIT 06 - CLEARWATER
drug CIRCUIT 17 - FT. LAUDERDALE
drug CIRCUIT 11 - MIAMI
robbery CIRCUIT 06 - CLEARWATER
robbery CIRCUIT 17 - FT. LAUDERDALE
robbery CIRCUIT 11 - MIAMI
burglary CIRCUIT 06 - CLEARWATER
burglary CIRCUIT 17 - FT. LAUDERDALE
burglary CIRCUIT 11 - MIAMI
driving CIRCUIT 06 - CLEARWATER
driving CIRCUIT 17 - FT. LAUDERDALE
driving CIRCUIT 11 - MIAMI


In [14]:
result_df = pd.concat(result_list)
#result_df

### aggregating results and apply multiple comparison adjustment

In [15]:

result_df_agg = pd.DataFrame(result_df.groupby(['area','crime']).mean()).reset_index()

In [16]:
from statsmodels.stats.multitest import multipletests

In [17]:
result_df_agg['if_reject'] = multipletests(result_df_agg['pvalue 2 sided'],method = 'bonferroni')[0]
result_df_agg['p_adjust'] = multipletests(result_df_agg['pvalue 2 sided'],method = 'bonferroni')[1]

In [18]:
result_df_agg

,area,crime,T statistic,df,pvalue 2 sided,Difference in mean,lb,ub,if_reject,p_adjust
0,CIRCUIT 06 - CLEARWATER,burglary,2.991724,31252.671865,1.379411e-02,29.485466,10.165794,48.805138,False,1.655293e-01
1,CIRCUIT 06 - CLEARWATER,driving,2.824543,31400.098775,1.792615e-02,27.699466,8.484873,46.914058,False,2.151138e-01
2,CIRCUIT 06 - CLEARWATER,drug,4.976834,8976.168877,1.638064e-05,56.283890,34.091374,78.476407,True,1.965677e-04
3,CIRCUIT 06 - CLEARWATER,robbery,4.302430,3923.352205,2.219247e-04,135.886884,73.929657,197.844112,True,2.663097e-03
4,CIRCUIT 11 - MIAMI,burglary,-3.339974,32512.583139,3.048760e-03,-35.100950,-55.710191,-14.491709,True,3.658512e-02
5,CIRCUIT 11 - MIAMI,driving,-3.421148,33460.945918,3.560551e-03,-35.470421,-55.806076,-15.134766,True,4.272661e-02
6,CIRCUIT 11 - MIAMI,drug,-8.919564,9588.713707,1.601592e-15,-110.615395,-134.945026,-86.285764,True,1.921911e-14
7,CIRCUIT 11 - MIAMI,robbery,1.248922,4124.543537,2.844143e-01,39.539382,-22.319496,101.398260,False,1.000000e+00
8,CIRCUIT 17 - FT. LAUDERDALE,burglary,2.498607,36892.631833,2.797441e-02,25.352027,5.479997,45.224058,False,3.356929e-01
9,CIRCUIT 17 - FT. LAUDERDALE,driving,2.422311,36091.222333,3.128229e-02,24.796660,4.740847,44.852473,False,3.753875e-01


In [19]:
result_df_agg.to_csv('circuit_equalStrata_bon.csv')

In [22]:
result_list = []

for i in crime_l:
    for j in circuit_l:
        print(i,j)
        df_data = sp_circuit_crime_type(circuit=j,crime_type=i,df= raw)
        df_model = cluster_prep(df_data)
        
        gen_male = df_model.loc[df_model['gender_ind']==1,:].copy()
        gen_female = df_model.loc[df_model['gender_ind']==0,:].copy()
        m = assign_label(gen_male)
        f = assign_label(gen_female)
        
        for k in range(1,101): ## instead of running once, run multiple test to provide average test stats
            
            
            b_m,w_m = cluster_sampling_unequal_strat(m)

            ind = 0

            if b_m.shape[0]==0 and w_m.shape[0] == 0:
                #print("not enough male samples")
                ind +=1


            b_f,w_f = cluster_sampling_unequal_strat(f)

            if b_f.shape[0]==0 and w_f.shape[0] == 0:
                #print("not enough female samples")
                ind +=1

            if ind == 2:
                #print('not enough samples to make inference')
                continue


            treat = pd.concat([b_m,b_f])
            cont = pd.concat([w_m,w_f])

            temp_result = welch_ttest(treat.sp_total_days, cont.sp_total_days,'unequal',area = j,crime = i)

            #print(temp_result)

            result_list.append(temp_result)

drug CIRCUIT 06 - CLEARWATER
drug CIRCUIT 17 - FT. LAUDERDALE
drug CIRCUIT 11 - MIAMI
robbery CIRCUIT 06 - CLEARWATER
robbery CIRCUIT 17 - FT. LAUDERDALE
robbery CIRCUIT 11 - MIAMI
burglary CIRCUIT 06 - CLEARWATER
burglary CIRCUIT 17 - FT. LAUDERDALE
burglary CIRCUIT 11 - MIAMI
driving CIRCUIT 06 - CLEARWATER
driving CIRCUIT 17 - FT. LAUDERDALE
driving CIRCUIT 11 - MIAMI


In [23]:
result_df = pd.concat(result_list)
result_df_agg = pd.DataFrame(result_df.groupby(['area','crime']).mean()).reset_index()

result_df_agg['if_reject'] = multipletests(result_df_agg['pvalue 2 sided'],method = 'bonferroni')[0]
result_df_agg['p_adjust'] = multipletests(result_df_agg['pvalue 2 sided'],method = 'bonferroni')[1]

In [24]:
result_df_agg.to_csv('circuit_unequalStrata_bon.csv')

In [25]:
result_df

,T statistic,df,pvalue 2 sided,Difference in mean,lb,ub,area,crime
0,8.184693,24132.200866,2.863587e-16,59.617184,45.340119,73.894248,CIRCUIT 06 - CLEARWATER,drug
0,8.099960,24510.708988,5.751683e-16,57.414724,43.521279,71.308170,CIRCUIT 06 - CLEARWATER,drug
0,8.049291,24410.693143,8.704121e-16,56.883138,43.031678,70.734597,CIRCUIT 06 - CLEARWATER,drug
0,8.281141,24254.364586,1.282284e-16,59.526925,45.437507,73.616343,CIRCUIT 06 - CLEARWATER,drug
0,8.045206,24097.081124,9.003792e-16,56.980389,43.098194,70.862584,CIRCUIT 06 - CLEARWATER,drug
...,...,...,...,...,...,...,...,...
0,-3.583399,55804.997318,3.394406e-04,-24.228569,-37.480834,-10.976305,CIRCUIT 11 - MIAMI,driving
0,-3.701335,55794.651744,2.146740e-04,-24.797592,-37.928918,-11.666266,CIRCUIT 11 - MIAMI,driving
0,-3.913442,55773.008801,9.109756e-05,-26.329594,-39.516495,-13.142693,CIRCUIT 11 - MIAMI,driving
0,-3.428264,55805.144699,6.078920e-04,-22.980648,-36.119139,-9.842157,CIRCUIT 11 - MIAMI,driving


In [26]:
county_l = ['pinellas','polk','monroe','escambia','volusia']
crime_l = ['drug','robbery','driving']

In [27]:
result_list = []

In [33]:
for i in crime_l:
    for j in county_l:
        print(i,j)
        df_data = sp_county_crime_type(county=j,crime_type=i,df= raw)
        df_model = cluster_prep(df_data)
        
        gen_male = df_model.loc[df_model['gender_ind']==1,:].copy()
        gen_female = df_model.loc[df_model['gender_ind']==0,:].copy()
        
        m = assign_label(gen_male)
        f = assign_label(gen_female)
        
        for k in range(1,101):
        
            b_m,w_m = cluster_sampling_unequal_strat(m)

            ind = 0

            if b_m.shape[0]==0 and w_m.shape[0] == 0:
                #print("not enough male samples")
                ind +=1


            b_f,w_f = cluster_sampling_unequal_strat(f)

            if b_f.shape[0]==0 and w_f.shape[0] == 0:
                #print("not enough female samples")
                ind +=1

            if ind == 2:
                #print('not enough samples to make inference')
                continue


            treat = pd.concat([b_m,b_f])
            cont = pd.concat([w_m,w_f])

            temp_result = welch_ttest(treat.sp_total_days, cont.sp_total_days,'unequal',area = j,crime = i)

            #print(temp_result)

            result_list.append(temp_result)


drug pinellas
drug polk
drug monroe
drug escambia
drug volusia
robbery pinellas
robbery polk
robbery monroe
robbery escambia
robbery volusia
driving pinellas
driving polk
driving monroe
driving escambia
driving volusia


In [34]:
result_df = pd.concat(result_list)
result_df_agg = pd.DataFrame(result_df.groupby(['area','crime']).mean()).reset_index()

result_df_agg['if_reject'] = multipletests(result_df_agg['pvalue 2 sided'],method = 'bonferroni')[0]
result_df_agg['p_adjust'] = multipletests(result_df_agg['pvalue 2 sided'],method = 'bonferroni')[1]

In [35]:
result_df_agg

,area,crime,T statistic,df,pvalue 2 sided,Difference in mean,lb,ub,if_reject,p_adjust
0,escambia,driving,2.676240,20175.735032,0.025107,34.139901,8.328086,59.951715,False,0.376612
1,escambia,drug,-1.136415,4224.051589,0.324782,-25.336886,-67.268503,16.594731,False,1.000000
2,escambia,robbery,4.472524,3975.288702,0.000291,129.712809,71.497413,187.928205,True,0.004365
3,monroe,driving,1.058237,2191.336933,0.337931,30.343165,-30.995956,91.682287,False,1.000000
4,monroe,drug,2.657024,1018.132408,0.058431,80.416856,17.951021,142.882690,False,0.876459
5,monroe,robbery,0.548373,118.296977,0.546187,29.510866,-82.507612,141.529345,False,1.000000
6,pinellas,driving,5.340274,44511.100558,0.000055,44.061271,27.293107,60.829435,True,0.000830
7,pinellas,drug,6.834357,14515.619460,0.000038,62.095246,43.265411,80.925081,True,0.000571
8,pinellas,robbery,4.819644,6088.291314,0.001030,124.151251,70.801275,177.501227,True,0.015456
9,polk,driving,1.510562,19984.771088,0.211268,18.894322,-7.263114,45.051758,False,1.000000


In [36]:
result_df_agg.to_csv('county_unequalStrata_bon.csv')